In [59]:
import pandas as pd
import numpy as np
import random
from functools import reduce
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
from konlpy.tag import *
hannanum = Hannanum()
kkma = Kkma()
twitter = Twitter()
comoran = Komoran()
mecab = Mecab()

In [35]:
from nltk import sent_tokenize

In [36]:
f = open('data/[Book 1] 해리포터와 마법사의돌 2-1-1.txt') 

In [37]:
sentences = f.read()
x = sent_tokenize(sentences)

In [96]:
def vocab(x):
    '''
    vocab 생성
    '''
    N = len(x) # numbers of sentences
    encoded = []
    for i in range(N):
        n = len(x[i])
        encoded.append(twitter.morphs(x[i]))
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    encoded = flatten(encoded)
    vect = CountVectorizer()
    vect.fit(encoded)
    encoded = vect.transform(x).toarray()
    print(vect.transform(x).toarray().shape) # 총 문장수 및 단어수
    return encoded

In [98]:
x0 = vocab(x)

(897, 2499)


# making files

In [176]:
f = open("data2/해리포터.txt")

In [177]:
x = sent_tokenize(f.read())

In [179]:
df = pd.DataFrame(x)

In [180]:
df.to_csv("harrypotter_line.csv", encoding="utf-8")

---

# model

In [48]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, Multiply
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
from nltk import sent_tokenize
import numpy as np
import re

Using TensorFlow backend.


In [1]:
from functools import reduce
from nltk import sent_tokenize
from konlpy.tag import *
comoran = Komoran()
twitter = Twitter()
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
sentences = pd.read_csv('harrypotter_line.csv').iloc[:, 1].tolist()

In [21]:
def vocab(x):
    '''
    one-hot-encoded과 vocab 생성
    '''
    N = len(x) # numbers of sentences
    encoded = []
    for i in range(N):
        n = len(x[i])
        encoded.append(twitter.morphs(x[i]))
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    encoded = flatten(encoded)
    vect = CountVectorizer()
    vect.fit(encoded)
    words = vect.vocabulary_
    encoded = vect.transform(x).toarray()
    print(vect.transform(x).toarray().shape) # 총 문장수 및 단어수
    N = 
    return encoded, words

In [37]:
N = len(x0) # numbers of sentences
encoded = []
for i in range(N):
    n = len(x0[i])
    encoded.append(twitter.morphs(x0[i]))
flatten = lambda data: reduce(lambda x, y: x + y, data)
encoded = flatten(encoded)
vect = CountVectorizer()
vect.fit(encoded)
words = vect.vocabulary_
encoded = vect.transform(x0).toarray()
print(vect.transform(x0).toarray().shape) # 총 문장수 및 단어수

(40, 182)


In [39]:
encoded, words = vocab(sentences) # (N-lines, vocab_size)

(6857, 8765)


# vectorize

In [37]:
chapter_idx =[-1, 
              244 -1, 
              573 -1, 
              1085 -1, 
              1689 -1, 
              2224 -1, 
              2667 -1, 
              3191 -1, 
              3626 -1, 
              4204 -1, 
              4697 -1, 
              5253 -1, 
              5771,]
labels = range(1, 13)

In [38]:
sentences_df = pd.read_csv("text.csv", header=None)
question_df = pd.read_csv("qa.csv", header=None, names=[0, 1, 2, "chapter"])

In [39]:
answer_df = question_df.iloc[:, [1]]

In [251]:
sentences_df["chapter"] = pd.cut(sentences_df.index, bins=chapter_idx, labels=labels)

In [252]:
pd.cut(sentences_df.index, bins=chapter_idx, labels=labels)

[1, 1, 1, 1, 1, ..., 12, 12, 12, 12, 12]
Length: 5772
Categories (12, int64): [1 < 2 < 3 < 4 ... 9 < 10 < 11 < 12]

In [253]:
any(sentences_df.isnull()[0] == True)

False

In [254]:
sentences_df[0][sentences_df[0].str.contains('\n') == True] = sentences_df[0][sentences_df[0].str.contains('\n') == True].str.replace('\n', '')

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [255]:
question_df.tail()

,0,1,2,chapter
492,말포이의 아버지가 골칫거리라고 생각하는 것은 누구입니까?,덤블도어,5666,12
493,지난 번에 비밀의 방이 열린 것은 몇년 전 입니까?,50년,5693,12
494,지난 번에 비밀의 방이 열렸을 때 죽은 것은 누구입니까?,잡종,5696,12
495,말포이의 집 마루 밑에 있는 것은 무엇입니까?,밀실,5718,12
496,헤르미온느의 폴리주스에 들어간 것은 누구의 털 입니까?,고양이,5760,12


In [256]:
any(question_df.isnull()[0] == True)

False

In [257]:
question_df[2][question_df["chapter"].isin([2])] = question_df[2][question_df["chapter"].isin([2])] - 2
question_df[2][question_df["chapter"].isin(range(3,13))] = question_df[2][question_df["chapter"].isin(range(3,13))] - 3

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [258]:
answer_df.tail()

,1
492,덤블도어
493,50년
494,잡종
495,밀실
496,고양이


In [259]:
any(answer_df.isnull()[1] == True)

True

In [260]:
answer_df[answer_df.isnull()[1]]

,1
340,NaN


In [261]:
data = pd.concat([sentences_df.iloc[:,0], question_df.iloc[:, 0], answer_df.iloc[:, 0]])

In [262]:
data = data.reset_index(drop=True)

In [263]:
data[data.isnull()] = " "

In [264]:
N = len(data)
encoded = []
for i in range(N):
    n = len(data[i])
    encoded.append(twitter.morphs(data.values[i]))
flatten = lambda data: reduce(lambda x, y: x + y, data)
encoded = flatten(encoded)
vect = CountVectorizer()
vect.fit(encoded)
words = vect.vocabulary_

In [265]:
V = len(words)

data 나누기

In [266]:
sentences_df["chapter"] = sentences_df["chapter"].astype(np.int)

In [267]:
for i in labels:
    df_temp = pd.concat([sentences_df.loc[sentences_df["chapter"].isin([i]), :],
                         question_df.loc[question_df["chapter"].isin([i]), :]])
    df_temp = df_temp.reset_index()
    idx_list = df_temp[2].unique().tolist()[1:]
    for j in idx_list:
        df_temp.loc[df_temp[2] == j, 2] = df_temp.index[df_temp["index"].isin([j])].values[0]
    df_temp = df_temp.drop("index", axis=1)
    df_temp = df_temp.reset_index()
    if i == 1:
        df = df_temp
    else:
        df = pd.concat([df, df_temp])

In [268]:
df = df.loc[:, ["index", 0, 1, 2, "chapter"]]

In [269]:
df = df.reset_index(drop= True)

In [270]:
df['index'] = df['index'] + 1

In [271]:
df.head()

,index,0,1,2,chapter
0,1,제 1장 최악의 생일.,NaN,NaN,1
1,2,"처음은 아니었지만, 프리벳가 4번지에서의 아침 식사시간은 말다툼으로 떠들썩했다.",NaN,NaN,1
2,3,버논 이모부가 조카 해리의 방에서 이른 아침부터 시끄럽게 울어대는 부엉이 소리 때문...,NaN,NaN,1
3,4,"""이번 주만도 벌써 세 번이다!""",NaN,NaN,1
4,5,그가 식탁을 앞에 두고 고함을 질렀다.,NaN,NaN,1


In [272]:
df.shape

(6269, 5)

In [273]:
df[1][df[1].isnull()] = ""

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [274]:
df[2][df[2].isnull()] = -1
df[2] = df[2].astype(np.int)

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [275]:
df[2][df[2] == -1] = ""

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [276]:
df['index'] = df['index'].astype(np.str)
df[1] = df[1].astype(np.str)
df[2] = df[2].astype(np.str)

In [277]:
df_new = df['index'] + " " + df.loc[:, 0] +  "\t" + df[1] + "\t" + df[2]

In [278]:
df_new

0                                      1 제 1장 최악의 생일.\t\t
1       2 처음은 아니었지만, 프리벳가 4번지에서의 아침 식사시간은 말다툼으로 떠들썩했다....
2       3 버논 이모부가 조카 해리의 방에서 이른 아침부터 시끄럽게 울어대는 부엉이 소리 ...
3                                4 "이번 주만도 벌써 세 번이다!"\t\t
4                             5 그가 식탁을 앞에 두고 고함을 질렀다.\t\t
5                   6 "저 부엉이를 어떻게 할 수 없다면, 당장 내다 버려!"\t\t
6                          7 해리는 그러나 다시 한번 해명하려고 애썼다.\t\t
7                                       8 "심심해서 그래요."\t\t
8                                           9 그가 말했다.\t\t
9          10 "바깥에서 이리저리 날아다니다가 집에만 갇혀 있으니깐 그러는 거라구요.\t\t
10                                  11 밤만이라도 나가게 하면."\t\t
11                                 12 "내가 멍청이인 줄 아냐?"\t\t
12      13 버논 이모부가 달걀 프라이가 조금 매달린 텁수룩한 콧수염을 들이대며 호통을 쳤...
13            14 "저 부엉이를 나가게 했다간 어떤 일이 벌어질지 안 봐도 뻔해."\t\t
14                      15 그는 아내 페투니아와 비밀스러운 눈길을 나눴다.\t\t
15      16 해리는 다시 설득해보려고 말을 꺼냈지만 그의 말은 더즐리 부부의 아들 두들리의...
16                                      17 "베이컨 더 줘."\t\t
17            

In [280]:
with open('file.txt' , 'w') as f:
    for i in range(len(df_new.index)):
        f.write(df_new[i] + '\n')

# test 

In [ ]:
test_data.csv

In [14]:
test_df = pd.read_csv("test_data.csv", header=None)
testq_df = pd.read_csv("test_q.csv", header=None)

In [16]:
test_df = test_df.loc[:, [1, 0]]

In [22]:
test_df.columns = [0, 4]

In [23]:
testq_df

,0,1,2,3
0,폼프리 부인이 헤르미온느가 창피당하지 않도록 침대에 달아준 것은 무엇입니까?,커튼,5778,13
1,해리와 론이 매일 헤르미온느를 찾아간 이유는 무엇입니까?,숙제,5780,13
2,헤르미온느의 얼굴에 난 것은 무엇입니까?,털,5781,13
3,헤르미온느의 베개에서 삐져 나온 황금빛 물건은 무엇입니까?,카드,5794,13
4,폼프리 부인이 헤르미온느를 위해 들고 온 것은 무엇입니까?,약,5801,13
5,필치가 밤새도록 해야하는 일은 무엇입니까?,걸레질,5811,13
6,머틀이 변기 파이프에 앉아 있을 때 머리 위로 떨어진 것은 무엇입니까?,책,5851,13
7,얇은 책의 표지는 무슨 색 입니까?,검정색,5852,13


In [24]:
test_df

,0,4
0,제 13장 비밀 일기.,5775
1,헤르미온느는 병동에 몇 주일을 머무렀다.,5776
2,"학생들이 크리스마스 휴일을 보내고 다시 돌아오자, 그녀가 습격을 받아서 병동에 입원...",5777
3,많은 학생들이 그 애를 한번 보려고 병동 앞을 지나다녔으므로 폼프리 부인은 헤르미온...,5778
4,해리와 론은 매일 저녁 그녀를 찾아갔다.,5779
5,그녀에게 매일 매일의 숙제를 알려주기 위해서 였다.,5780
6,"""만약 내 얼굴에 털이 자라났다면,난 공부하지 않고 쉬었을 거야.""",5781
7,어느 날 저녁 론이 헤르미온느의 머리맡 탁자 위에 책들을 쏟아내며 말했다.,5782
8,"""바보 같은 소리 마,론,그때 그때 해놓지 않으면 나중엔 따라갈 수가 없어.""",5783
9,헤르미온느가 활발하게 말했다.,5784


In [39]:
df = pd.concat([test_df, testq_df])
df = df.reset_index(drop=True)

In [40]:
df.tail(10)

,0,1,2,3,4
77,너덜너덜한 검정색 표지를 갖고 있었는데 다른 것들과 마찬가지로 축 젖어 있었다.,NaN,NaN,NaN,5852.0
78,"해리가 그것을 집으려고 한 발짝 내딛었을 때,론이 그의 등짝을 덥석 잡았다.",NaN,NaN,NaN,5853.0
79,폼프리 부인이 헤르미온느가 창피당하지 않도록 침대에 달아준 것은 무엇입니까?,커튼,5778.0,13.0,NaN
80,해리와 론이 매일 헤르미온느를 찾아간 이유는 무엇입니까?,숙제,5780.0,13.0,NaN
81,헤르미온느의 얼굴에 난 것은 무엇입니까?,털,5781.0,13.0,NaN
82,헤르미온느의 베개에서 삐져 나온 황금빛 물건은 무엇입니까?,카드,5794.0,13.0,NaN
83,폼프리 부인이 헤르미온느를 위해 들고 온 것은 무엇입니까?,약,5801.0,13.0,NaN
84,필치가 밤새도록 해야하는 일은 무엇입니까?,걸레질,5811.0,13.0,NaN
85,머틀이 변기 파이프에 앉아 있을 때 머리 위로 떨어진 것은 무엇입니까?,책,5851.0,13.0,NaN
86,얇은 책의 표지는 무슨 색 입니까?,검정색,5852.0,13.0,NaN


In [41]:
idx_list = df[2].unique().tolist()[1:]

In [42]:
idx_list

[5778.0, 5780.0, 5781.0, 5794.0, 5801.0, 5811.0, 5851.0, 5852.0]

In [45]:
for j in idx_list:
    df.loc[df[2] == j, 2] = df.index[df[4].isin([j])].values[0]

In [47]:
df.tail()

,0,1,2,3,4
82,헤르미온느의 베개에서 삐져 나온 황금빛 물건은 무엇입니까?,카드,19.0,13.0,NaN
83,폼프리 부인이 헤르미온느를 위해 들고 온 것은 무엇입니까?,약,26.0,13.0,NaN
84,필치가 밤새도록 해야하는 일은 무엇입니까?,걸레질,36.0,13.0,NaN
85,머틀이 변기 파이프에 앉아 있을 때 머리 위로 떨어진 것은 무엇입니까?,책,76.0,13.0,NaN
86,얇은 책의 표지는 무슨 색 입니까?,검정색,77.0,13.0,NaN


In [48]:
df = df.reset_index()

In [50]:
df['index'] = df['index'] + 1

In [51]:
df

,index,0,1,2,3,4
0,1,제 13장 비밀 일기.,NaN,NaN,NaN,5775.0
1,2,헤르미온느는 병동에 몇 주일을 머무렀다.,NaN,NaN,NaN,5776.0
2,3,"학생들이 크리스마스 휴일을 보내고 다시 돌아오자, 그녀가 습격을 받아서 병동에 입원...",NaN,NaN,NaN,5777.0
3,4,많은 학생들이 그 애를 한번 보려고 병동 앞을 지나다녔으므로 폼프리 부인은 헤르미온...,NaN,NaN,NaN,5778.0
4,5,해리와 론은 매일 저녁 그녀를 찾아갔다.,NaN,NaN,NaN,5779.0
5,6,그녀에게 매일 매일의 숙제를 알려주기 위해서 였다.,NaN,NaN,NaN,5780.0
6,7,"""만약 내 얼굴에 털이 자라났다면,난 공부하지 않고 쉬었을 거야.""",NaN,NaN,NaN,5781.0
7,8,어느 날 저녁 론이 헤르미온느의 머리맡 탁자 위에 책들을 쏟아내며 말했다.,NaN,NaN,NaN,5782.0
8,9,"""바보 같은 소리 마,론,그때 그때 해놓지 않으면 나중엔 따라갈 수가 없어.""",NaN,NaN,NaN,5783.0
9,10,헤르미온느가 활발하게 말했다.,NaN,NaN,NaN,5784.0


In [52]:
df[1][df[1].isnull()] = ""

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [53]:
df[2][df[2].isnull()] = -1
df[2] = df[2].astype(np.int)
df[2][df[2] == -1] = ""

/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [55]:
df['index'] = df['index'].astype(np.str)
df[1] = df[1].astype(np.str)
df[2] = df[2].astype(np.str)

In [56]:
df_new = df['index'] + " " + df.loc[:, 0] +  "\t" + df[1] + "\t" + df[2]

In [57]:
df_new

0                                   1 제 13장  비밀 일기.\t\t
1                          2 헤르미온느는 병동에 몇 주일을 머무렀다.\t\t
2     3 학생들이 크리스마스 휴일을 보내고 다시 돌아오자, 그녀가 습격을 받아서 병동에 ...
3     4 많은 학생들이 그 애를 한번 보려고 병동 앞을 지나다녔으므로 폼프리 부인은 헤르...
4                          5 해리와 론은 매일 저녁 그녀를 찾아갔다.\t\t
5                    6 그녀에게 매일 매일의 숙제를 알려주기 위해서 였다.\t\t
6           7 "만약 내 얼굴에 털이 자라났다면,난 공부하지 않고 쉬었을 거야."\t\t
7       8 어느 날 저녁 론이 헤르미온느의 머리맡 탁자 위에 책들을 쏟아내며 말했다.\t\t
8     9 "바보 같은 소리 마,론,그때 그때 해놓지 않으면 나중엔 따라갈 수가 없어."\t\t
9                               10 헤르미온느가 활발하게 말했다.\t\t
10    11 이제 얼굴에 난 털이 모두 사라지고 눈이 서서히 갈색으로 돌아가고 있었으므로 ...
11                       12 "그런데 무슨 새로운 실마리라도 잡았니?"\t\t
12              13 그녀가 폼프리 부인이 들을 수 없도록 작은 소리로 말했다.\t\t
13                                         14 "전혀."\t\t
14                                 15 해리가 침울하게 말했다.\t\t
15                       16 "분명히 말포이 짓일 거라고 생각했는데."\t\t
16                        17 론이 100번도 더 했던 말을 또 했다.\t\t
17                                      18 "저건 뭐

In [58]:
with open('file_test.txt' , 'w') as f:
    for i in range(len(df_new.index)):
        f.write(df_new[i] + '\n')

In [2]:
f = open('file_train.txt')

In [3]:
x = f.readlines()

In [4]:
x[1]

'2 처음은 아니었지만, 프리벳가 4번지에서의 아침 식사시간은 말다툼으로 떠들썩했다.\n'

In [5]:
comoran.morphs(x[1])

['2',
 '처음',
 '은',
 '아니',
 '었',
 '지만',
 ',',
 '프리벳가',
 '4',
 '번지',
 '에서',
 '의',
 '아침 식사',
 '시간',
 '은',
 '말다툼',
 '으로',
 '떠들썩',
 '하',
 '었',
 '다',
 '.']

In [11]:
temp = comoran.pos(x[-2])

In [12]:
temp

[('78', 'SN'),
 ('너덜너덜', 'MAG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETM'),
 ('검정색', 'NNG'),
 ('표지', 'NNG'),
 ('를', 'JKO'),
 ('갖', 'VV'),
 ('고', 'EC'),
 ('있', 'VX'),
 ('었', 'EP'),
 ('는데', 'EC'),
 ('다른', 'MM'),
 ('것', 'NNB'),
 ('들', 'XSN'),
 ('과', 'JC'),
 ('마찬가지', 'NNG'),
 ('로', 'JKB'),
 ('축', 'MAG'),
 ('젖', 'VV'),
 ('어', 'EC'),
 ('있', 'VX'),
 ('었', 'EP'),
 ('다', 'EF'),
 ('.', 'SF')]

In [28]:
temp = twitter.pos(x[-1])

In [29]:
temp

[('79', 'Number'),
 ('해리', 'Noun'),
 ('가', 'Josa'),
 ('그것', 'Noun'),
 ('을', 'Josa'),
 ('집으', 'Verb'),
 ('려고', 'Eomi'),
 ('한', 'Verb'),
 ('발짝', 'Noun'),
 ('내딛', 'Verb'),
 ('었', 'PreEomi'),
 ('을', 'Eomi'),
 ('때', 'Noun'),
 (',', 'Punctuation'),
 ('론', 'Noun'),
 ('이', 'Josa'),
 ('그', 'Noun'),
 ('의', 'Josa'),
 ('등짝', 'Noun'),
 ('을', 'Josa'),
 ('덥석', 'Noun'),
 ('잡았', 'Verb'),
 ('다', 'Eomi'),
 ('.', 'Punctuation')]

In [13]:
j_list = ["JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ", "JC", "JX", 
          "ETM", "ETN", "EP", "EF", "EC", "XPN", "XSN", "XSV", "XSA"]

In [30]:
j_list = ["Josa", "PreEomi", "Eomi"]

In [31]:
words = []
for i in range(len(temp)):
    if not(temp[i][1] in j_list):
        words.append(temp[i][0])

In [32]:
words

['79',
 '해리',
 '그것',
 '집으',
 '한',
 '발짝',
 '내딛',
 '때',
 ',',
 '론',
 '그',
 '등짝',
 '덥석',
 '잡았',
 '.']

In [44]:
y = question_df[1].values.tolist()

In [57]:
comoran.pos(y[8])

[('볼드모트', 'NNP')]

In [19]:
import re

In [24]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [21]:
tokenize(x[-1])

/Users/Soo/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['79',
 '해리가',
 '그것을',
 '집으려고',
 '한',
 '발짝',
 '내딛었을',
 '때',
 ',',
 '론이',
 '그의',
 '등짝을',
 '덥석',
 '잡았다',
 '.']

In [23]:
def tokenize2(sent):
    comoran = Komoran()
    j_list = ["JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ", "JC", "JX", 
          "ETM", "ETN", "EP", "EF", "EC", "XPN", "XSN", "XSV", "XSA"]
    temp = comoran.pos(sent)
    words = []
    for i in range(len(temp)):
        if not(temp[i][1] in j_list):
            words.append(temp[i][0])
    return words

In [26]:
x

['1 제 1장 최악의 생일.\n',
 '2 처음은 아니었지만, 프리벳가 4번지에서의 아침 식사시간은 말다툼으로 떠들썩했다.\n',
 '3 버논 이모부가 조카 해리의 방에서 이른 아침부터 시끄럽게 울어대는 부엉이 소리 때문에 잠에서 깬 것이다.\n',
 '4 이번 주만도 벌써 세 번이다!\n',
 '5 그가 식탁을 앞에 두고 고함을 질렀다.\n',
 '6 저 부엉이를 어떻게 할 수 없다면, 당장 내다 버려!\n',
 '7 해리는 그러나 다시 한번 해명하려고 애썼다.\n',
 '8 심심해서 그래요.\n',
 '9 그가 말했다\n',
 '10 바깥에서 이리저리 날아다니다가 집에만 갇혀 있으니깐 그러는 거라구요.\n',
 '11 밤만이라도 나가게 하면.\n',
 '12 내가 멍청이인 줄 아냐?\n',
 '13 버논 이모부가 달걀 프라이가 조금 매달린 텁수룩한 콧수염을 들이대며 호통을 쳤다.\n',
 '14 저 부엉이를 나가게 했다간 어떤 일이 벌어질지 안 봐도 뻔해.\n',
 '15 그는 아내 페투니아와 비밀스러운 눈길을 나눴다.\n',
 '16 해리는 다시 설득해보려고 말을 꺼냈지만 그의 말은 더즐리 부부의 아들 두들리의 시끄럽고 긴 트림소리 때문에 들리지 않았다.\n',
 '17 베이컨 더 줘.\n',
 '18 프라이팬에 있단다. 얘야.\n',
 '19 페투니아 이모가 그 뚱보 아들을 눈물이 그렁그렁한 표정으로 바라보며 말했다.\n',
 '20 집에 있는 동안 잘 먹여야지.\n',
 '21 학교 음식이 영 시원치 않아서 말야.\n',
 '22 당치않은 소리 말아요, 페투니아.\n',
 '23 내가 스멜팅스에 있을 때는 굶주린 적이 한번도 없었다구.\n',
 '24 버논 이모부가 힘차게 말했다.\n',
 '25 두들리는 먹고 싶은 만큼 춘분히 먹을 거야, 안 그러니 얘야?\n',
 '26 식탁 의자 옆으로 엉덩이가 축 늘어질 정도로 살이 뒤룩뒤룩 찐 두들 리가 씩 웃으며 해리 쪽으로 고개를 돌렸다.\n',
 '27 프라이팬 좀 이리 줘.\n',
 '28 너 주문을

In [33]:
def tokenize3(sent):
    twitter = Twitter()
    j_list = j_list = ["Josa", "PreEomi", "Eomi"]
    temp = twitter.pos(sent)
    words = []
    for i in range(len(temp)):
        if not(temp[i][1] in j_list):
            words.append(temp[i][0])
    return words

In [35]:
x[1].split(" ", 1)

['2', '처음은 아니었지만, 프리벳가 4번지에서의 아침 식사시간은 말다툼으로 떠들썩했다.\n']

In [ ]:
nid, line = line.split(' ', 1)